In [1]:
import pandas as pd
import numpy as np

In [8]:
train = pd.read_csv("data/train.csv")
train.shape

(15289, 18)

In [17]:
train.head()

,id,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds,yield
0,0,25.0,0.50,0.25,0.75,0.50,69.7,42.1,58.2,50.2,24.3,41.2,24.0,0.39,0.425011,0.417545,32.460887,4476.81146
1,1,25.0,0.50,0.25,0.50,0.50,69.7,42.1,58.2,50.2,24.3,41.2,24.0,0.39,0.444908,0.422051,33.858317,5548.12201
2,2,12.5,0.25,0.25,0.63,0.63,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.552927,0.470853,38.341781,6869.77760
3,3,12.5,0.25,0.25,0.63,0.50,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.565976,0.478137,39.467561,6880.77590
4,4,25.0,0.50,0.25,0.63,0.63,77.4,46.8,64.7,55.8,27.0,45.8,24.0,0.39,0.579677,0.494165,40.484512,7479.93417


In [25]:
train.columns

Index(['id', 'clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia',
       'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange',
       'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange',
       'RainingDays', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds',
       'yield'],
      dtype='object')

In [18]:
train.clonesize.value_counts()

25.0    8245
12.5    6717
37.5     265
20.0      56
10.0       4
40.0       2
Name: clonesize, dtype: int64

In [19]:
train.honeybee.value_counts()

0.500     7832
0.250     7285
0.750      110
0.537       38
0.000       16
18.430       5
6.640        3
Name: honeybee, dtype: int64

In [20]:
train.bumbles.value_counts()

0.250    10856
0.380     4376
0.117       39
0.000        5
0.042        3
0.058        2
0.065        2
0.585        2
0.293        2
0.560        1
0.260        1
Name: bumbles, dtype: int64

In [21]:
train.andrena.value_counts()

0.380    4565
0.500    4165
0.630    3042
0.750    1828
0.250    1624
0.409      43
0.000       8
0.229       4
0.490       2
0.147       2
0.707       1
0.240       1
0.560       1
0.101       1
0.234       1
0.235       1
Name: andrena, dtype: int64

In [23]:
train.osmia.value_counts()

0.630    4763
0.500    4699
0.750    4387
0.250     872
0.380     509
0.058      42
0.000       6
0.021       4
0.117       2
0.620       1
0.585       1
0.020       1
0.078       1
0.606       1
Name: osmia, dtype: int64

In [30]:
#定义X，y
X = train.iloc[:,1:-1]
y = train['yield']

In [32]:
y

0        4476.81146
1        5548.12201
2        6869.77760
3        6880.77590
4        7479.93417
            ...    
15284    7667.83619
15285    3680.56025
15286    4696.44394
15287    6772.93347
15288    5867.99722
Name: yield, Length: 15289, dtype: float64

In [50]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

num_cols = ['fruitset', 'fruitmass', 'seeds']

oe_cols = ['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia',
       'MaxOfUpperTRange', 'MinOfUpperTRange', 'AverageOfUpperTRange',
       'MaxOfLowerTRange', 'MinOfLowerTRange', 'AverageOfLowerTRange',
       'RainingDays', 'AverageRainingDays']

scaler = StandardScaler()
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

preprocessor = make_column_transformer(
    (scaler, num_cols),
    (oe, oe_cols))

In [1]:
from sklearn.linear_model import Lasso,Ridge,SGDRegressor,LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,VotingRegressor,HistGradientBoostingRegressor,RandomForestRegressor

lasso = Lasso() #370.70
ridge = Ridge() #371.22
svr = SVR() #882.06
sgd = SGDRegressor() #391
dtr = DecisionTreeRegressor() #530.42
gbr = GradientBoostingRegressor() #354.89
lr = LinearRegression() #371.39
rft = RandomForestRegressor()
hgbr = HistGradientBoostingRegressor() #355.18


cv1 = VotingRegressor([('gbr',gbr),('lr',lr)]) #357.08
cv2 = VotingRegressor([('gbr',gbr),('hgbr',hgbr)]) #353.02

NameError: name 'lgb' is not defined

In [116]:
# gbr = GradientBoostingRegressor(n_estimators=80) #354.89
# # hgbr = HistGradientBoostingRegressor() #355.18
# # cv = VotingRegressor([('gbr',gbr),('hgbr',hgbr)]) #353.02

In [120]:
#make pipeline
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(preprocessor, cv2)

In [121]:
pipe.steps

[('columntransformer',
  ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                   ['fruitset', 'fruitmass', 'seeds']),
                                  ('ordinalencoder',
                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                  unknown_value=-1),
                                   ['clonesize', 'honeybee', 'bumbles', 'andrena',
                                    'osmia', 'MaxOfUpperTRange',
                                    'MinOfUpperTRange', 'AverageOfUpperTRange',
                                    'MaxOfLowerTRange', 'MinOfLowerTRange',
                                    'AverageOfLowerTRange', 'RainingDays',
                                    'AverageRainingDays'])])),
 ('votingregressor',
  VotingRegressor(estimators=[('gbr', GradientBoostingRegressor()),
                              ('hgbr', HistGradientBoostingRegressor())]))]

In [122]:
#cv score baseline
from sklearn.model_selection import cross_val_score
print(cross_val_score(pipe, X, y, scoring='neg_mean_absolute_error').mean())

-353.2038937980527


In [ ]:
# from sklearn.model_selection import GridSearchCV
# params = {}
# params['gradientboostingregressor__loss'] = ['squared_error','absolute_error','huber','quantile']
# params['gradientboostingregressor__laerning_rate'] = [0.1,0.05,0.01]
# params['gradientboostingregressor__n_estimator'] = [100,300,500]

In [123]:
pipe.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['fruitset', 'fruitmass',
                                                   'seeds']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['clonesize', 'honeybee',
                                                   'bumbles', 'andrena',
                                                   'osmia', 'MaxOfUpperTRange',
                                                   'MinOfUpperTRange',
                                                   'AverageOfUpperTRange',
                                                   'MaxOfLow

In [124]:
test = pd.read_csv('data/test.csv')
X_test = test.drop(columns=['id'])
X_test.head()

,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds
0,25.0,0.25,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.399367,0.408088,31.394569
1,12.5,0.25,0.25,0.75,0.63,94.6,57.2,79.0,68.2,33.0,55.9,1.0,0.10,0.488048,0.442866,36.846956
2,12.5,0.25,0.25,0.63,0.63,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.583379,0.487057,40.037644
3,25.0,0.50,0.38,0.38,0.63,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.433014,0.422847,33.116091
4,37.5,0.75,0.25,0.25,0.25,94.6,57.2,79.0,68.2,33.0,55.9,24.0,0.39,0.360996,0.388860,29.558019


In [125]:
result = pipe.predict(X_test)

In [126]:
result

array([4268.95775617, 6131.60874792, 7184.33377655, ..., 6498.66075742,
       4406.68142365, 7313.06382061])

In [128]:
pd.DataFrame({'id':test.id,'yield':result}).set_index('id').to_csv('gradient_bosting_regressor_ver1.csv')

- 353.13521